# Bayes Theorem for Predicting the Probability of an Email Being Spam

S = Spam
w = Word

$P(Spam|w_{1}, w_{2},..., w_{n}) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_{i}|Spam)$

The probability that an email consisting of the words $w_{1}, w_{2},... w_{n}$ is proportional to the probability that any given email is spam multiplied by the product of each word's probability to appear in a spam email.



In [23]:
import pandas as pd
import math

pd.options.mode.chained_assignment = None  # default='warn'

PREDICTION = 'Prediction'
CLASSIFICATION = 'Classiciation'

## Functions

In [2]:
def count_vocab(emails):
    total_words = 0
    
    for index, row in emails.iterrows():
        total_words += sum(row.values[1:-2])
            
    return total_words

In [3]:
def calculate_word_spamicity(w_spam_count, vocab, spam_vocab):
    alpha = 1
    
    spamicity = (w_spam_count + alpha) / (spam_vocab + alpha * vocab)
    return spamicity

In [4]:
def build_word_spamicity_dict(spam_emails, vocab, spam_vocab):
    spam_word_appearances = {}
    
    for (column_name, column_data) in spam_emails.iteritems():
        if column_name != 'Email No.' and column_name != PREDICTION and column_name != CLASSIFICATION:
            spam_word_appearances[column_name] = sum(column_data.values)
            
    for word in spam_word_appearances:
        spam_word_appearances[word] = calculate_word_spamicity(spam_word_appearances[word], vocab, spam_vocab)
            
    return spam_word_appearances

In [5]:
def calculate_emails(testing_emails, word_spamicities, spam_proportion):
    test_data_map = {}
    for index, email in testing_emails.iterrows():
        email_spamicity = math.log(spam_proportion)
        for column in testing_data.columns[1:-2]:
            if email[column] > 0:
                # log P(spam) + sum log P(w|spam)
                email_spamicity += math.log(word_spamicities[column])*email[column]

        test_data_map[index] = email_spamicity
        
    return test_data_map

In [6]:
def calculate_accuracy(testing_emails):
    number_correct = 0
    for index, email in testing_emails.iterrows():
        if email[PREDICTION] == email[CLASSIFICATION]:
            number_correct += 1
        
    return number_correct / testing_data.shape[0] * 100

# Model

## Step 1: Partition the data into training and test segments

20% of the data for testing, and the remaining 80% is training (i.e. the 80% training data will confirm whether the 20% testing data labels are correct).

In [21]:
df = pd.read_csv('emails.csv')
total_num_emails = df.shape[0]

end = total_num_emails//5

testing_data = df.iloc[:end].copy()

# This is where the model's prediction will be stored
testing_data[CLASSIFICATION] = ""

training_data = df.iloc[end:]

testing_data


,Email No.,the,to,ect,and,for,of,a,you,hou,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction,Classiciation
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,1,0,0,
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Email 1030,10,6,4,3,5,2,45,10,0,...,0,0,0,0,0,0,0,0,0,
1030,Email 1031,8,19,18,7,3,3,150,8,11,...,0,0,2,0,0,0,2,0,0,
1031,Email 1032,0,0,1,0,0,2,7,0,0,...,0,0,0,0,0,0,0,0,0,
1032,Email 1033,1,2,1,4,1,2,108,5,0,...,0,0,0,0,0,0,2,0,1,


In [8]:
training_data

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
1034,Email 1035,18,19,18,18,3,6,143,3,0,...,0,0,0,0,0,0,0,3,0,0
1035,Email 1036,4,5,3,0,2,2,19,0,0,...,0,0,0,0,0,0,0,0,0,0
1036,Email 1037,0,1,1,1,0,0,6,1,0,...,0,0,0,0,0,0,0,0,0,1
1037,Email 1038,3,2,1,1,0,1,11,0,0,...,0,0,0,0,0,0,0,1,0,1
1038,Email 1039,11,7,11,11,16,0,98,3,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,1,0,1


## Step 2: Get probabilities that any one email in the training data is either spam or ham

In the labelled dataset, count the number of spam and ham emails.

$P(Spam) = \frac{Spam\,Emails}{Total\,Emails}$

In [9]:
spam_proportion = training_data['Prediction'].value_counts()[1] / training_data.shape[0]
spam_proportion

0.291445142580957

$P(Spam) = \frac{Ham\,Emails}{Total\,Emails}$

In [10]:
ham_proportion = training_data['Prediction'].value_counts()[0] / training_data.shape[0]
ham_proportion

0.708554857419043

## Step 3: Get the "spamicity" probability of each word in the testing data email

**w** = word
<br>**vocab** = total words in dataset
<br>**spam_vocab**
<br>**wi_spam_count**

Count all unique words in the labelled dataset to get **vocab**.

Count the total number of words in labelled spam emails (ignoring uniqueness) to get **spam_vocab**.

For each word **w**, count all instances of the word in the spam emails to get **wi_spam_count**.

Calculate spamicity of each word and store the word and its spamicity in a dictionary

$P(w_{i}|Spam) = \frac{wi\_spam\_count\,+\,\alpha}{spam\_vocab\,+\,\alpha \cdot vocab}$

$\alpha$ is a coefficient that prevents a probability from being 0.


In [11]:
# Subtract 3 for "Email No.", "Prediction", and "Classification" columns
total_vocab = len(training_data.columns) - 3
total_vocab

2999

In [12]:
spam_training_emails = training_data.loc[training_data[PREDICTION] == 1]

total_spam_words = count_vocab(spam_training_emails)
total_spam_words

1708339

In [13]:
ham_training_emails = training_data.loc[training_data[PREDICTION] == 0]

total_ham_words = count_vocab(ham_training_emails)
total_ham_words

2972320

In [14]:
word_spamicities = build_word_spamicity_dict(spam_training_emails, total_vocab, total_spam_words)
word_spamicities

{'the': 0.004709765107769476,
 'to': 0.004884482200477053,
 'ect': 0.0017904119466756422,
 'and': 0.0029468170519207776,
 'for': 0.002195358251847385,
 'of': 0.0032769680799468017,
 'a': 0.04756745891226631,
 'you': 0.0023706596826576633,
 'hou': 0.00043766923892299474,
 'in': 0.010783375347242918,
 'on': 0.008480498884498562,
 'is': 0.004917205134228306,
 'this': 0.0011488087099100236,
 'enron': 5.843381027009276e-07,
 'i': 0.04614985467511386,
 'be': 0.0025786840472191933,
 'that': 0.0006702358037979639,
 'will': 0.0003278136756152204,
 'have': 0.0004663018059553402,
 'with': 0.0009834410268456612,
 'your': 0.0009711699266889416,
 'at': 0.006023357162641162,
 'we': 0.0013743632175525816,
 's': 0.03882400788155233,
 'are': 0.0011079050427209587,
 'it': 0.004733722969980214,
 'by': 0.0004213077720473688,
 'com': 0.002026484540166817,
 'as': 0.0033026789564656427,
 'from': 0.0004832476109336671,
 'gas': 0.00013381342551851242,
 'or': 0.00656386990763952,
 'not': 0.0007380190237112716,
 

In [15]:
word_hamicities = build_word_spamicity_dict(ham_training_emails, total_vocab, total_ham_words)
word_hamicities

{'the': 0.006328397055912324,
 'to': 0.005621245990766032,
 'ect': 0.005653175340190413,
 'and': 0.002541912312595725,
 'for': 0.0029838817283121574,
 'of': 0.0017594752024908925,
 'a': 0.04759489654722737,
 'you': 0.0019540761847721203,
 'hou': 0.002342942050919582,
 'in': 0.008279448354949502,
 'on': 0.00987154654677364,
 'is': 0.004440868357308914,
 'this': 0.0012143235733714603,
 'enron': 0.001892906273243306,
 'i': 0.035427797826048234,
 'be': 0.003014130585661571,
 'that': 0.0008580592534783665,
 'will': 0.0009430921524717182,
 'have': 0.0008247855103940115,
 'with': 0.0007226115922359922,
 'your': 0.0005414545465545039,
 'at': 0.005894830100570729,
 'we': 0.0019376073624374395,
 's': 0.03448470567357651,
 'are': 0.0013427131678989716,
 'it': 0.0033999715660740916,
 'by': 0.0006271596423778425,
 'com': 0.001365231761703535,
 'as': 0.004627739076045291,
 'from': 0.0008523455804234773,
 'gas': 0.0008110054753792786,
 'or': 0.006628532940501506,
 'not': 0.000727989166875888,
 'me': 

## Step 4: Calculate the "spamicity" of the email

Multiply spamicities of each word together to get $\prod_{i=1}^{n}P(w_{i}|Spam)$.

Multiply that product by the probability that any email is spam.

In [16]:
test_data_spam_map = calculate_emails(testing_data, word_spamicities, spam_proportion)
test_data_spam_map
        

{0: -266.9075221940168,
 1: -11065.705573601223,
 2: -612.2982729725492,
 3: -5162.3888490623085,
 4: -5402.578731317851,
 5: -6200.146920417468,
 6: -3315.163995618576,
 7: -2675.8484879136818,
 8: -2153.7278385576774,
 9: -5977.984864458365,
 10: -10448.280709749228,
 11: -18800.780967609604,
 12: -11507.05188412098,
 13: -5252.924869682581,
 14: -3055.112809985521,
 15: -3641.922129284476,
 16: -999.0987661392444,
 17: -20578.67992253658,
 18: -1920.5732493069534,
 19: -4287.189301732568,
 20: -1241.314382993542,
 21: -3725.7987341217686,
 22: -4225.503421527134,
 23: -1619.8405849291198,
 24: -878.480509370133,
 25: -22867.856522265665,
 26: -6453.803364500153,
 27: -1980.1940051895547,
 28: -9741.48704019633,
 29: -10470.837945562558,
 30: -3128.357852014049,
 31: -824.4444803298838,
 32: -22082.538859980268,
 33: -6166.595204439004,
 34: -4621.698091101696,
 35: -2686.8743001291555,
 36: -6836.3044559395785,
 37: -2322.488142121421,
 38: -2759.006804923557,
 39: -5470.87590723463

In [17]:
test_data_ham_map = calculate_emails(testing_data, word_hamicities, ham_proportion)
test_data_ham_map

{0: -263.75211820145074,
 1: -10742.205577671839,
 2: -571.4301285088811,
 3: -4908.086120785854,
 4: -5109.256370217733,
 5: -6318.8333059031675,
 6: -3208.113639397985,
 7: -2770.371884010296,
 8: -2093.2435212150745,
 9: -5746.3492132693145,
 10: -10175.688614963581,
 11: -18411.78530983217,
 12: -11346.55494784154,
 13: -5061.996686136625,
 14: -2890.2807573195687,
 15: -3568.9286404061227,
 16: -993.1663314379663,
 17: -20833.795254004108,
 18: -1847.7364922576032,
 19: -4063.397946448457,
 20: -1190.604549724376,
 21: -3613.9217752488908,
 22: -3905.7883379698173,
 23: -1576.78738253868,
 24: -822.2968228419828,
 25: -23440.470640653086,
 26: -6268.8951861496935,
 27: -1905.6001643369555,
 28: -9549.308201114809,
 29: -10187.154828162067,
 30: -3095.659160868708,
 31: -844.8024625923059,
 32: -21319.977070346587,
 33: -5918.072218666594,
 34: -4480.585182041,
 35: -2593.1789632442046,
 36: -6650.237039748687,
 37: -2210.443814714411,
 38: -2707.4515589863418,
 39: -5526.071793287

In [28]:
for key in test_data_spam_map:
    if test_data_spam_map[key] >= test_data_ham_map[key]:
        testing_data[CLASSIFICATION].loc[testing_data.index[key]] = 1
    else:
        testing_data[CLASSIFICATION].loc[testing_data.index[key]] = 0
        
testing_data

,Email No.,the,to,ect,and,for,of,a,you,hou,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction,Classiciation
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,1,0,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Email 1030,10,6,4,3,5,2,45,10,0,...,0,0,0,0,0,0,0,0,0,0
1030,Email 1031,8,19,18,7,3,3,150,8,11,...,0,0,2,0,0,0,2,0,0,0
1031,Email 1032,0,0,1,0,0,2,7,0,0,...,0,0,0,0,0,0,0,0,0,0
1032,Email 1033,1,2,1,4,1,2,108,5,0,...,0,0,0,0,0,0,2,0,1,1


In [29]:
score = calculate_accuracy(testing_data)
f'Accuracy: {score}%'

'Accuracy: 93.90715667311412%'

## Step 5: Repeat steps 2-4 calculating the "hamicity" of each email

## Step 6: Label emails in test dataset and compare

A probability greater then 0.5 will indicate whether an email is ham or spam.